# Hill Cypher Encryption
---

In this example, we'll be using a linear transformation (and its inverse) to encode and decode a string of lowercase characters. This logic is currently part of how the Amazon Launch Service encrypts/encodes source tokens when an Ibotta Saver starts up an Amazon shopping session!

If you're new to `.ipynb` notebooks, make sure to run *all* cells here to make sure you have all the imports/tools you need available.

In [1]:
# importing numpy for linear algebra operations, scipy for sparse matrices
import numpy as np
from scipy.sparse import coo_matrix

## Contents
---
[1 Integer representation](#1-Integer-representation)  
[2 Encoding one letter](#2-Encoding-one-letter)  
[3 Encoding all letters](#3-Encoding-all-letters)  
[4 Encoding all letters simultaneously](#4-Encoding-all-letters-simultaneously)  
[5 Decoding one letter](#5-Decoding-one-letter)  
[6 Decoding all the letters](#6-Decoding-all-the-letters)  

## 1 Integer representation
---
Our goal here will be to take the string "ibotta" and encrypt it via a linear transformation. To do this, we'll first map each character in the lowercase string to the integer range [0, 25] (where a = 0, b = 1, ..., z = 25). Here is the result of that mapping:

```
i -> 8
b -> 1
o -> 14
t -> 19
t -> 19
a -> 0
```

We'll use these values to populate an integer vector representation of the string. I'm going to initialize a (6,1) numpy matrix to do this. Note the transpose ('T') used to convert the default interpretation of a *row* vector/matrix to the *column* format we want.

In [2]:
messageIntVec0 = np.matrix([8, 1, 14, 19, 19, 0]).T
print(messageIntVec0)

[[ 8]
 [ 1]
 [14]
 [19]
 [19]
 [ 0]]


[Back to top](#Hill-Cypher-Encryption)

## 2 Encoding one letter
---
There are more general ways of constructing Hill cyphers/transformations than what we'll explore today (and you can read more at https://en.wikipedia.org/wiki/Hill_cipher). For our illustration, though, we'll use a simple composition that'll allow us to easily determine a forward (encryption) and inverse (decryption) transformation.

Let's say we want to use the secret key 'mykey' to encrypt the information. Here's one way we can do that.

First, as we did with the 'ibotta' string, let's transform the 'mykey' string into an integer array representation.

```
m -> 12
y -> 24
k -> 10
e -> 4
y -> 24
```

I'm actually going to keep these values as a *row* vector because of how they'll be subsequently used.

In [3]:
keyIntArray = np.array([12, 24, 10, 4, 24])

We'll use these values to start transforming the letters of our 'ibotta' message, one by one. Below is the construction (using sparse matrix logic as a 'helper') of a matrix operator that only transforms the *last* element in our 'ibotta' integer vector.

In [4]:
row    = np.array([5, 5, 5, 5, 5])
col    = np.array([0, 1, 2, 3, 4])
values = keyIntArray

firstTransformation = coo_matrix((values, (row, col)), shape = (6, 6)).toarray() + np.identity(6)
print(firstTransformation)

[[  1.   0.   0.   0.   0.   0.]
 [  0.   1.   0.   0.   0.   0.]
 [  0.   0.   1.   0.   0.   0.]
 [  0.   0.   0.   1.   0.   0.]
 [  0.   0.   0.   0.   1.   0.]
 [ 12.  24.  10.   4.  24.   1.]]


From what we know about matrix-vector multiplication, the action of this matrix on our message integer vector results in the first five elements being left alone, with the sixth (representing the letter 'a' in 'ibotta') being replaced by its integer representation (0 in the case of 'a') *plus a linear combination of the other integers in the vector*.

In [5]:
print(firstTransformation*messageIntVec0)

[[   8.]
 [   1.]
 [  14.]
 [  19.]
 [  19.]
 [ 792.]]


The nice thing about our application of linear algebra is that we're interested in transformations between integers *modulo 26*. When reflecting upon how the rest of our mathematics is going to work (have faith for the moment! :) ) we're okay recording results modulo 26 after each operation if we like, e.g.:

In [6]:
messageIntVec1 = firstTransformation*messageIntVec0 % 26
print(messageIntVec1)

[[  8.]
 [  1.]
 [ 14.]
 [ 19.]
 [ 19.]
 [ 12.]]


Taking results modulo 26 also allows us to interpret this result with a transformed *string* representation:

```
 8 -> i
 1 -> b
14 -> o
19 -> t
19 -> t
12 -> m
```

So now our string representation reads 'ibottm'. We can continue similarly - from this result - to encode each of the other letters of our message in turn.

[Back to top](#Hill-Cypher-Encryption)

## 3 Encoding all letters
---
What we'll do now is generalize the pattern we used to construct the first transformation of our message's integer representation in such a way that we can transform *all* the letters of the message.

As we proceed, we'll see how we can multiply individual transformation matrices together into a *single* encryption matrix that transforms all the letters at once (and is equivalent to transforming each individual integer representation in succession).

For our second transformation, we'll encode the fifth character of our six letter string, using a 'rotation' of the transformation we employed first on the *final* character of the string.

In [7]:
row    = np.array([4, 4, 4, 4, 4])
col    = np.array([5, 0, 1, 2, 3])
values = keyIntArray

secondTransformation = coo_matrix((values, (row, col)), shape = (6, 6)).toarray() + np.identity(6)
print(secondTransformation)

[[  1.   0.   0.   0.   0.   0.]
 [  0.   1.   0.   0.   0.   0.]
 [  0.   0.   1.   0.   0.   0.]
 [  0.   0.   0.   1.   0.   0.]
 [ 24.  10.   4.  24.   1.  12.]
 [  0.   0.   0.   0.   0.   1.]]


*This* transformation is expressing the *fifth* 'letter-equivalent-integer' as its current entry *plus* a linear combination of the other entries currently in the string (importantly, we note that we're using this on the string that's already been acted upon by the first transformation).

In [8]:
messageIntVec2 = secondTransformation*messageIntVec1 % 26
print(messageIntVec2)

[[  8.]
 [  1.]
 [ 14.]
 [ 19.]
 [ 19.]
 [ 12.]]


Welp, even though there's been a transformation of the 5th character, it's hard to tell; during this transformation the fifth character was replaced by:

```
character 5 = (24*8 + 10*1 + 4*14 + 24*19 + 1*19 + 12*12) % 26 = 19
```
...so just by chance, we've got another 't' in the fifth position.

Now, I'm going to speed things up a bit and carry out a similar process for the fourth, third, second, and first character in our string (in that order). [*Note: there're certainly more efficient, less hardcoded ways to code this, but since we're dealing with a small array size and introducing a new topic, I'll keep things very expanded and explicit.*]

In [9]:
row    = np.array([3, 3, 3, 3, 3])
col    = np.array([4, 5, 0, 1, 2])
values = keyIntArray

thirdTransformation = coo_matrix((values, (row, col)), shape = (6, 6)).toarray() + np.identity(6)

row    = np.array([2, 2, 2, 2, 2])
col    = np.array([3, 4, 5, 0, 1])
values = keyIntArray

fourthTransformation = coo_matrix((values, (row, col)), shape = (6, 6)).toarray() + np.identity(6)

row    = np.array([1, 1, 1, 1, 1])
col    = np.array([2, 3, 4, 5, 0])
values = keyIntArray

fifthTransformation = coo_matrix((values, (row, col)), shape = (6, 6)).toarray() + np.identity(6)

row    = np.array([0, 0, 0, 0, 0])
col    = np.array([1, 2, 3, 4, 5])
values = keyIntArray

sixthTransformation = coo_matrix((values, (row, col)), shape = (6, 6)).toarray() + np.identity(6)

encodedIntVec = (sixthTransformation*(fifthTransformation*(fourthTransformation*(thirdTransformation*messageIntVec2)))) % 26
print(encodedIntVec)

[[ 10.]
 [ 13.]
 [ 16.]
 [ 19.]
 [ 19.]
 [ 12.]]


The string form of our encoded message is now:

```
10 -> k
13 -> n
16 -> q
19 -> t
19 -> t
12 -> m
```

or 'knqttm'.

[Back to top](#Hill-Cypher-Encryption)

## 4 Encoding all letters simultaneously
---
Another way we can go about the transformation above in one fell swoop is to take all the transformations that went into encoding our integer vector representation and multiply them all together first, in the following order (note that the '@' symbol is a special signifier for numpy to perform traditional (mathematical) matrix multiplication):

In [10]:
compositeTransformation = sixthTransformation @ \
                            fifthTransformation @ \
                            fourthTransformation @ \
                            thirdTransformation @ \
                            secondTransformation @ \
                            firstTransformation % 26
                    
print(compositeTransformation)

[[ 19.  24.   6.  12.  20.  14.]
 [ 14.   7.   0.  22.   8.  22.]
 [ 22.  10.  25.  14.  12.   0.]
 [  0.  22.  10.  25.  14.  12.]
 [ 12.  12.  20.  20.   3.  12.]
 [ 12.  24.  10.   4.  24.   1.]]


We can now use this composite transformation to encode our string in *one* multiplication, with the same 'knqttm' result as above (where we transformed each letter separately):

In [11]:
print(compositeTransformation*messageIntVec0 % 26)

[[ 10.]
 [ 13.]
 [ 16.]
 [ 19.]
 [ 19.]
 [ 12.]]


[Back to top](#Hill-Cypher-Encryption)

## 5 Decoding one letter
---
Let's take a look at the *final* (sixth) transformation that transforms our very first character.

In [12]:
print(sixthTransformation)

[[  1.  12.  24.  10.   4.  24.]
 [  0.   1.   0.   0.   0.   0.]
 [  0.   0.   1.   0.   0.   0.]
 [  0.   0.   0.   1.   0.   0.]
 [  0.   0.   0.   0.   1.   0.]
 [  0.   0.   0.   0.   0.   1.]]


After the fifth transformation is complete, this sixth-and-final transformation expresses the first integer as itself, plus twelve times the second integer, plus 24 times the third, etc. All we'd have to do in order to *reverse* this process is to take the first entry, **subtract** 12 times the second integer, **subtract** 24 times the third integer, and so on. In matrix form, we'll visualize this *inverse* transformation as `sixthInverse`:

In [13]:
sixthInverse = -sixthTransformation + 2*np.identity(6)
print(sixthInverse)

[[  1. -12. -24. -10.  -4. -24.]
 [  0.   1.   0.   0.   0.   0.]
 [  0.   0.   1.   0.   0.   0.]
 [  0.   0.   0.   1.   0.   0.]
 [  0.   0.   0.   0.   1.   0.]
 [  0.   0.   0.   0.   0.   1.]]


Note how similar this is to the structure of the `sixthTransformation` transformation that *encodes* the first integer. Also, note what results when we multiply the `sixthInverse` and the `sixthTransformation`:

In [14]:
print(sixthInverse @ sixthTransformation)

[[ 1.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  1.]]


This is the 6-by-6 identity matrix! As we expected (by construction) above, the inverse has 'undone' what the corresponding transformation did to our vector. Here's another way of looking at it - let's multiply the `sixthInverse` into the final `encodedIntVec`:

In [15]:
print(sixthInverse * encodedIntVec % 26)

[[  8.]
 [ 13.]
 [ 16.]
 [ 19.]
 [ 19.]
 [ 12.]]


We've successfully recovered the first letter of our original string 'ibotta' (remembering `8 -> i`)!

[Back to top](#Hill-Cypher-Encryption)

## 6 Decoding all the letters
---
After following along up until now, it might be apparent what we have to do - starting from the encoded integer vector - to get all the way back to our initial 'ibotta' string integer representation. We've got to create transformations that invert the fifth, fourth, third, etc. transformations and 'hit' the encoded vector with those inverses in the correct order.

In [16]:
fifthInverse  = -fifthTransformation + 2*np.identity(6)
fourthInverse = -fourthTransformation + 2*np.identity(6)
thirdInverse  = -thirdTransformation + 2*np.identity(6)
secondInverse = -secondTransformation + 2*np.identity(6)
firstInverse  = -firstTransformation + 2*np.identity(6)

recoveredIntVec = firstInverse @ \
                    secondInverse @ \
                    thirdInverse @ \
                    fourthInverse @ \
                    fifthInverse @ \
                    sixthInverse @ encodedIntVec % 26
                    
print(recoveredIntVec)

[[  8.]
 [  1.]
 [ 14.]
 [ 19.]
 [ 19.]
 [  0.]]


And there we go!

```
 8 -> i
 1 -> b
14 -> o
19 -> t
19 -> t
 0 -> a
```

As with the *forward* (encoding) transformation, we can also 'composite' all the inverse operations together into a *single* inverse matrix that reverses our encoding:

In [17]:
compositeInverse = firstInverse @ secondInverse @ thirdInverse @ fourthInverse @ fifthInverse @ sixthInverse % 26
print(compositeInverse)

[[  1.  14.   2.  16.  22.   2.]
 [  2.   3.  18.   8.   8.   0.]
 [  0.   2.   3.  18.   8.   8.]
 [  8.   8.  18.   1.  12.  24.]
 [ 24.   6.   4.  12.   9.   8.]
 [  8.   6.  22.   2.   6.  25.]]


In [18]:
print(compositeInverse * encodedIntVec % 26)

[[  8.]
 [  1.]
 [ 14.]
 [ 19.]
 [ 19.]
 [  0.]]


As a final note, look at what happens when we multiply the `compositeInverse` and `compositeTransformation` together...they completely negate each others' effect (producing the 6-by-6 identity matrix, at least when we're still talking about the integer-mod-26 field):

In [19]:
print(compositeInverse @ compositeTransformation % 26)

[[ 1.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  1.]]


[Back to top](#Hill-Cypher-Encryption)